<a href="https://colab.research.google.com/github/anhhuyluong/DS101_Project/blob/main/RainPredictionUsingNaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORT NECESSARY LIBRARIES AND MODULES**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = '/content/drive/My Drive/DS101/cleaned_weatherAUS.csv'

df = pd.read_csv(data_path)
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Day
0,2008-12-01,Albury,13.4,22.9,0.6,5.116752,8.5775,W,44.0,W,...,1007.1,8.0,5.0,16.9,21.8,No,No,2008,12,1
1,2008-12-02,Albury,7.4,25.1,0.0,5.116752,8.5775,WNW,44.0,NNW,...,1007.8,7.0,5.0,17.2,24.3,No,No,2008,12,2
2,2008-12-03,Albury,12.9,25.7,0.0,5.116752,8.5775,WSW,46.0,W,...,1008.7,7.0,2.0,21.0,23.2,No,No,2008,12,3
3,2008-12-04,Albury,9.2,28.0,0.0,5.116752,8.5775,NE,24.0,SE,...,1012.8,7.0,5.0,18.1,26.5,No,No,2008,12,4
4,2008-12-05,Albury,17.5,32.3,1.0,5.116752,8.5775,W,41.0,ENE,...,1006.0,7.0,8.0,17.8,29.7,Yes,No,2008,12,5


In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()
df['Day'] = df['Date'].dt.day_name()
df['Quarter'] = df['Date'].dt.quarter


df.drop(columns='Date', inplace=True)
df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Day,Quarter
0,Albury,13.4,22.9,0.6,5.116752,8.5775,W,44.0,W,WNW,...,8.0,5.0,16.9,21.8,No,No,2008,December,Monday,4
1,Albury,7.4,25.1,0.0,5.116752,8.5775,WNW,44.0,NNW,WSW,...,7.0,5.0,17.2,24.3,No,No,2008,December,Tuesday,4
2,Albury,12.9,25.7,0.0,5.116752,8.5775,WSW,46.0,W,WSW,...,7.0,2.0,21.0,23.2,No,No,2008,December,Wednesday,4
3,Albury,9.2,28.0,0.0,5.116752,8.5775,NE,24.0,SE,E,...,7.0,5.0,18.1,26.5,No,No,2008,December,Thursday,4
4,Albury,17.5,32.3,1.0,5.116752,8.5775,W,41.0,ENE,NW,...,7.0,8.0,17.8,29.7,Yes,No,2008,December,Friday,4


In [ ]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,Year,Quarter
count,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.00000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000,140787.000000
mean,12.188119,23.235811,2.349974,4.948942,8.094868,39.880922,13.88494,18.561241,68.920646,51.510928,1017.680503,1015.279178,4.753723,4.752481,16.967368,21.726381,2012.757492,2.468559
std,6.400488,7.112066,8.465173,2.210017,2.868427,13.154116,8.57018,8.778564,19.051760,20.708291,6.759875,6.694194,2.468789,2.260246,6.504388,6.963631,2.542272,1.114187
min,-8.500000,-4.800000,0.000000,0.300000,0.000000,6.000000,0.00000,0.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-7.200000,-5.400000,2007.000000,1.000000
25%,7.600000,17.900000,0.000000,3.600000,7.500000,31.000000,7.00000,13.000000,57.000000,37.000000,1013.500000,1011.000000,3.000000,3.000000,12.200000,16.600000,2011.000000,1.000000
50%,12.000000,22.600000,0.000000,5.116752,8.577500,38.192376,13.00000,19.000000,70.000000,52.000000,1017.978981,1015.525185,4.907658,4.904279,16.700000,21.100000,2013.000000,2.000000
75%,16.800000,28.200000,0.800000,5.800000,9.500000,46.000000,19.00000,24.000000,83.000000,66.000000,1021.800000,1019.400000,7.000000,7.000000,21.600000,26.500000,2015.000000,3.000000
max,33.900000,48.100000,371.000000,9.100000,14.500000,135.000000,37.00000,87.000000,100.000000,100.000000,1041.000000,1039.600000,9.000000,9.000000,40.200000,46.700000,2017.000000,4.000000


In [ ]:
df.dtypes

,0
Location,object
MinTemp,float64
MaxTemp,float64
Rainfall,float64
Evaporation,float64
Sunshine,float64
WindGustDir,object
WindGustSpeed,float64
WindDir9am,object
WindDir3pm,object


In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns
categorical_cols

Index(['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday',
       'RainTomorrow', 'Month', 'Day'],
      dtype='object')

**ENCODING CATEGORICAL COLUMNS**

In [ ]:
# Encoding categorical_cols
ordinal_encoder = OrdinalEncoder()
df[categorical_cols] = ordinal_encoder.fit_transform(df[categorical_cols])

df.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Day,Quarter
0,2.0,13.4,22.9,0.6,5.116752,8.5775,13.0,44.0,13.0,14.0,...,8.0,5.0,16.9,21.8,0.0,0.0,2008,2.0,1.0,4
1,2.0,7.4,25.1,0.0,5.116752,8.5775,14.0,44.0,6.0,15.0,...,7.0,5.0,17.2,24.3,0.0,0.0,2008,2.0,5.0,4
2,2.0,12.9,25.7,0.0,5.116752,8.5775,15.0,46.0,13.0,15.0,...,7.0,2.0,21.0,23.2,0.0,0.0,2008,2.0,6.0,4
3,2.0,9.2,28.0,0.0,5.116752,8.5775,4.0,24.0,9.0,0.0,...,7.0,5.0,18.1,26.5,0.0,0.0,2008,2.0,4.0,4
4,2.0,17.5,32.3,1.0,5.116752,8.5775,13.0,41.0,1.0,7.0,...,7.0,8.0,17.8,29.7,1.0,0.0,2008,2.0,0.0,4


In [ ]:
df.dtypes

,0
Location,float64
MinTemp,float64
MaxTemp,float64
Rainfall,float64
Evaporation,float64
Sunshine,float64
WindGustDir,float64
WindGustSpeed,float64
WindDir9am,float64
WindDir3pm,float64


In [ ]:
df = df.astype('float64')
print(df.dtypes)

Location         float64
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir      float64
WindGustSpeed    float64
WindDir9am       float64
WindDir3pm       float64
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday        float64
RainTomorrow     float64
Year             float64
Month            float64
Day              float64
Quarter          float64
dtype: object


**FEATURES SELECTION AND SPLITING THE DATASETS**

In [ ]:
X = df.drop('RainTomorrow', axis=1)
y = df['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

* **NAIVE BAYES METHOD**

In [ ]:
class NaiveBayes:
    def __init__(self, smoothing=1.0):
        self.smoothing = smoothing
        self.priors = None
        self.means = None
        self.variances = None

    def fit(self, X, y):
        classes = np.unique(y)
        self.priors = np.zeros(len(classes))
        self.means = np.zeros((len(classes), X.shape[1]))
        self.variances = np.zeros((len(classes), X.shape[1]))

        for i, c in enumerate(classes):
            X_c = X[y == c]
            self.priors[i] = (X_c.shape[0] + self.smoothing) / (X.shape[0] + self.smoothing * len(classes))
            self.means[i] = np.mean(X_c, axis=0)
            self.variances[i] = np.var(X_c, axis=0) + self.smoothing

    def predict_proba(self, X):
        probabilities = np.zeros((X.shape[0], len(self.priors)))
        for i, c in enumerate(self.priors):
            probabilities[:, i] = np.log(self.priors[i]) + np.sum(np.log(self.gaussian_pdf(X, self.means[i], self.variances[i])), axis=1)
        return probabilities

    def predict(self, X):
        probabilities = self.predict_proba(X)
        return np.argmax(probabilities, axis=1)

    def gaussian_pdf(self, x, mu, sigma):
        return 1 / (np.sqrt(2 * np.pi * sigma)) * np.exp(-0.5 * (x - mu) ** 2 / sigma)

In [ ]:
nb_model = NaiveBayes()
nb_model.fit(X_train, y_train)

In [ ]:
y_pred = nb_model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
confusion_mtx = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion matrix\n: {confusion_mtx}')

Accuracy: 0.812628737836494
Confusion matrix
: [[28900  3920]
 [ 3994  5423]]


**PREDICT FROM A NEW ROW OF DATA**

In [ ]:
new_data = pd.DataFrame([
    ['Albury', 13.4, 22.9, 0.6, 5.116751918158568, 8.577499999999999, 'W' , 44.0, 'W', 'WNW', 20.0, 24.0, 71.0, 22.0, 1007.7, 1007.1, 8.0, 5.0, 16.9, 21.8, 0.0, 1.0, 2020, 10.0, 15.0, 3]
], columns=df.columns)

new_data[categorical_cols] = ordinal_encoder.transform(new_data[categorical_cols])

# Predict probabilities
probabilities = nb_model.predict_proba(new_data)

# Extract probabilities
rain_prob = probabilities[0][1]
no_rain_prob = probabilities[0][0]

print("Probability of rain:", rain_prob)
print("Probability of no rain:", no_rain_prob)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''